In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv("main_database_large_diffs_draws_or_not.csv")

In [3]:
df.head(2)

,season,target,PosDiff,GSPGDiff,GCPGDiff,CSPGDiff,PtsPGDiff,SPGDiff,STPGDiff,S%Diff,PTSPG3Diff,PTSPG5Diff,PTSPG7Diff,GD3Diff,GD5Diff,GD7Diff,PosLSDiff,SpentDiff,IncomeDiff,NetDiff
0,9,0,-15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,38.23,-3.330,-41.560
1,9,1,-10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.99,6.129,-5.861


In [4]:
df['target'].value_counts()

0    1355
1     388
Name: target, dtype: int64

In [5]:
1355/388

3.4922680412371134

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

def get_train_test_split(data):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(data, data["target"]):
        data_train = data.loc[train_index]
        data_test = data.loc[test_index]
    return data_train, data_test

train,test = get_train_test_split(df)

X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [7]:
X_train.head(2)

,season,PosDiff,GSPGDiff,GCPGDiff,CSPGDiff,PtsPGDiff,SPGDiff,STPGDiff,S%Diff,PTSPG3Diff,PTSPG5Diff,PTSPG7Diff,GD3Diff,GD5Diff,GD7Diff,PosLSDiff,SpentDiff,IncomeDiff,NetDiff
1126,19,-13.0,0.22,-0.48,0.04,0.64,2.257,2.145,-0.012,-1.000,0.0,0.143,-2.000,-1.0,-0.286,-13.0,9.70,-1.82,-11.52
1166,20,11.0,-1.64,0.17,-0.21,-0.97,-3.556,-3.306,-0.050,-0.333,-0.4,-0.571,0.333,-0.2,-1.572,6.0,-49.59,11.10,60.69


In [8]:
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)
X_train_norm[0]

array([ 6.17873841e-01, -4.22755786e-01,  7.15432868e-03, -1.56094444e-02,
        1.30078703e-03,  2.08125925e-02,  7.33969084e-02,  6.97547047e-02,
       -3.90236110e-04, -3.25196758e-02,  0.00000000e+00,  4.65031365e-03,
       -6.50393517e-02, -3.25196758e-02, -9.30062729e-03, -4.22755786e-01,
        3.15440856e-01, -5.91858100e-02, -3.74626666e-01])

In [9]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(2, activation = tf.nn.softmax))

In [10]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

class_weight = {0: 1,
                1: 3.492}

In [11]:
model.fit(X_train_norm, y_train.values, epochs=20, validation_data=(X_test_norm,y_test), class_weight=class_weight)

Train on 1394 samples, validate on 349 samples
Epoch 1/20
1394/1394 [==============================] - 1s 850us/step - loss: 0.7029 - acc: 0.3651 - val_loss: 0.7054 - val_acc: 0.4155
Epoch 2/20
1394/1394 [==============================] - 0s 239us/step - loss: 0.6949 - acc: 0.4742 - val_loss: 0.6860 - val_acc: 0.4928
Epoch 3/20
1394/1394 [==============================] - 0s 154us/step - loss: 0.6913 - acc: 0.5380 - val_loss: 0.6806 - val_acc: 0.5358
Epoch 4/20
1394/1394 [==============================] - 0s 146us/step - loss: 0.6893 - acc: 0.5746 - val_loss: 0.6781 - val_acc: 0.5559
Epoch 5/20
1394/1394 [==============================] - 0s 165us/step - loss: 0.6886 - acc: 0.5760 - val_loss: 0.6750 - val_acc: 0.5645
Epoch 6/20
1394/1394 [==============================] - 0s 280us/step - loss: 0.6875 - acc: 0.5911 - val_loss: 0.6730 - val_acc: 0.5645
Epoch 7/20
1394/1394 [==============================] - 0s 265us/step - loss: 0.6863 - acc: 0.5925 - val_loss: 0.6693 - val_acc: 0.5759
E

In [12]:
train_pred = model.predict_classes(X_train_norm)
test_pred = model.predict_classes(X_test_norm)

import numpy as np

unique, counts = np.unique(train_pred, return_counts=True)
print("preds: " + str(dict(zip(unique, counts))))

unique, counts = np.unique(y_train.values, return_counts=True)
print("real: " + str(dict(zip(unique, counts))))

from sklearn.metrics import confusion_matrix as conf_matrix

print(conf_matrix(train_pred,y_train.values,labels=[0,1]))

preds: {0: 767, 1: 627}
real: {0: 1084, 1: 310}
[[629 138]
 [455 172]]


In [13]:
print(conf_matrix(test_pred,y_test.values))

[[151  35]
 [120  43]]


In [14]:
test_pred_probs = model.predict(X_test_norm)

In [15]:
test_pred_probs[0]

array([0.50514877, 0.4948512 ], dtype=float32)

In [ ]:
2184/1264 #1

In [ ]:
2184/1269 #0